<a href="https://colab.research.google.com/github/AleemKhan12/AleemKhan12/blob/calendar_data_algorithm/Attendance_Tracking_Calendar_Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [571]:
#Attendance Tracking by using Calendar and Audit Log Data

#Revision notes V0.8: Created new schema (Schema 2) for easier visualization 

'''
ISSUES
1.

'''


'\nISSUES\n1.\n\n'

# CONNECTING TO GOOGLE DRIVE

In [572]:
#connect colab to drive

from google.colab import drive
from oauth2client.client import GoogleCredentials
from google.colab import auth
import gspread
drive.mount('/content/gdrive')
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [573]:
#Imports
import pandas as pd
from datetime import date, timedelta
import pytz
from numpy.core.numeric import NaN
import datetime
import math



In [574]:
pytz.all_timezones

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Asmera',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti',
 'Africa/Douala',
 'Africa/El_Aaiun',
 'Africa/Freetown',
 'Africa/Gaborone',
 'Africa/Harare',
 'Africa/Johannesburg',
 'Africa/Juba',
 'Africa/Kampala',
 'Africa/Khartoum',
 'Africa/Kigali',
 'Africa/Kinshasa',
 'Africa/Lagos',
 'Africa/Libreville',
 'Africa/Lome',
 'Africa/Luanda',
 'Africa/Lubumbashi',
 'Africa/Lusaka',
 'Africa/Malabo',
 'Africa/Maputo',
 'Africa/Maseru',
 'Africa/Mbabane',
 'Africa/Mogadishu',
 'Africa/Monrovia',
 'Africa/Nairobi',
 'Africa/Ndjamena',
 'Africa/Niamey',
 'Africa/Nouakchott',
 'Africa/Ouagadougou',
 'Africa/Porto-Novo',
 'Africa/Sao_Tome',
 'Africa/Timbuktu',
 'Africa/

#IMPORTANT GLOBAL VARIABLES

In [575]:
mentor_email = 'mentor@villagebookbuilders.org' #Mentor program email address
oracle_data       = 0                           #Data read from Oracle Database SQL Query
google_data       = 0                           #Data read from Google Audit Log
active_sessions   = 0                           #Active sessions recorded in Oracle db based on end date
total_mentors     = 0                           #List of all mentors names in oracle db
oracle_mentor_emails = []                       #list of all mentor emails in oracle db
active_mentors    = 0                           #Mentor names that joined at least one session 
active_mentor_emails = []                       #List of mentor emails that joined at least one session
mentor_programs   = []                          #Names of all mentor programs stored in oracle database
mentors_not_registered = []                     #Names of all active mentors not logged in oracle database
time_zone = 'America/Port_of_Spain'                         #google_data['Date'][1][len(google_data['Date'][1])-3:] #TODO: change this hard coded time zone
start = 0                                       #start date
end = 0                                         #end date
program_director_name = ['adeiso ghana', 'ahero kenya', 'biwi, malawi vbb mentor program', 'charity centre, zambia vbb mentor program', 'chebosi kenya', 'kadzakalowa malawi', 'maragoli kenya', 'mukono uganda', 'vima ghana'] #Actor names of program director accounts

# READING DATA SOURCES

In [576]:
#Read Data Sources as pandas data frames

#Calendar Data
calendar_path = '/content/gdrive/MyDrive/Projects/Mentor_Dashboard_V2/3. Execution/SRC Code/Data/Calendar Data Export/Calendar Export UTC - Sheet1.csv' 
calendar_data = pd.read_csv(calendar_path)

#Google Data
#google_path = '/content/gdrive/MyDrive/Projects/Mentor_Dashboard_V2/3. Execution/SRC Code/Data/Google Audit Log/Jan 2022/Copy of meet_logs_1643126905783 - meet_logs_1643126905783.csv'
google_path = '/content/gdrive/MyDrive/Projects/Mentor_Dashboard_V2/3. Execution/SRC Code/Data/Google Audit Log/March 2022/3_5_2022.csv'
google_data = pd.read_csv(google_path)


#Mentee Computer Emails
mentee_email_path = '/content/gdrive/MyDrive/Projects/Mentor_Dashboard_V2/3. Execution/SRC Code/Data/mentee email/Copy of Village Library Information - computer email.csv'
mentee_computer_emails = pd.read_csv(mentee_email_path)





# CLEANING DATA

In [577]:

#Calendar Data
calendar_data = calendar_data.dropna() #remove all empty sessions

#Google Data
google_data = google_data.dropna(subset=['Date', 'Meeting code', 'Participant Identifier', 'Organizer email', 'Duration', 'Actor name', 'Calendar event ID']) #clean google data
google_data = google_data[google_data['Duration'] > 0] #drop all calls with 0 duration
google_data = google_data[google_data['Organizer email'] == mentor_email] #only consider mentor sessions
google_data = google_data[google_data['Actor name'] != 'Mentor Program'] #drop all entries for mentor program

#Mentee Email data
mentee_computer_emails = mentee_computer_emails.dropna()

In [578]:
print(google_data.shape, calendar_data.shape)

(9482, 64) (8065, 11)


In [579]:
#DATE CONVERSIONS
#Only run this once. Running more times will cause an error. To fix re-read data sources
def conv_to_datetime(a):
  utc_timezone = pytz.timezone('UTC')
  if isinstance(a, datetime.date):
    return a
  try:
    #Convert calendar export dates
    date = datetime.datetime.strptime(a, '%m/%d/%Y %H:%M:%S')
    date = date.replace(tzinfo = pytz.timezone('UTC')) 
    return date
  except:
    try:
      #Convert Audit log dates 
      date = datetime.datetime.strptime(a, '%b %d, %Y, %I:%M:%S %p')
      date = date.replace(tzinfo = pytz.timezone(time_zone)) 
      date = date.astimezone(utc_timezone) #convert all times to UTC
      return  date
    except:
      date = datetime.datetime.strptime('', '')
      date = date.replace(tzinfo = utc_timezone)
      return date



In [580]:
#converting oracle dates to datetime objs
calendar_data['Event Start UTC'] = calendar_data['Event Start UTC'].apply(conv_to_datetime)
calendar_data['Event End UTC'] = calendar_data['Event End UTC'].apply(conv_to_datetime)



In [581]:
#Convert google dates to datetime UTC
google_data['Date'] = google_data['Date'].apply(lambda x: x[:len(x)-4]) #remove time zone
google_data['Date'] = google_data['Date'].apply(conv_to_datetime)

In [582]:
#Create new column with just dates
google_data['String_Date'] = google_data['Date'].apply(lambda x: x.strftime("%m/%d/%Y"))
calendar_data['String_Date'] = calendar_data['Event Start UTC'].apply(lambda x: x.strftime("%m/%d/%Y"))

In [583]:
#lower participant emails for accurate searches
google_data['Participant Identifier'] = google_data['Participant Identifier'].apply(lambda x: x.lower())
calendar_data['Participant Identifier'] = calendar_data['Participant Identifier'].apply(lambda x: x.lower())

In [584]:
#Drop duplicate sessions caused by people signing in from various devices at the same time
google_data = google_data.drop_duplicates(subset = ['Participant Identifier', 'Calendar event ID', 'String_Date'])

In [585]:
#list of mentee computer emails
mentee_computer_emails_list = mentee_computer_emails['COMPUTER_EMAIL'].tolist()

#DATE RANGES

In [586]:
#Today's date
today_date = datetime.datetime.today().replace(tzinfo = pytz.timezone(time_zone))

#Start and end dates for schema 1
dates_list = sorted(google_data['Date'].tolist())
start1, end1 = dates_list[0], dates_list[-1] 

#Start and end dates for schema 2
end, start = datetime.datetime(2022, 2, 27).astimezone(pytz.timezone("utc")), datetime.datetime(2022, 3, 5).astimezone(pytz.timezone("utc"))



In [587]:
print(start1, end1)
print(start, end)

2021-09-07 11:09:41+00:00 2022-03-05 18:26:28+00:00
2022-03-05 00:00:00+00:00 2022-02-27 00:00:00+00:00


#ATTENDANCE AND DURATION TRACKING 

In [588]:
calendar_data['Attendance'] = [0]*len(calendar_data)
calendar_data['Duration'] = [0]*len(calendar_data)
calendar_data['User Type'] = [0]*len(calendar_data)

In [ ]:
for index, meeting in calendar_data.iterrows():
  attendance = 0
  duration = 0
  meeting_data = google_data[(google_data['Participant Identifier'] == meeting['Participant Identifier']) & (google_data['Calendar event ID'].str.contains(meeting['Calendar event ID'])) & (google_data['String_Date'] == meeting['String_Date'])]

  #print(meeting_data)

  if (len(meeting_data) > 0):
    attendance = 1
    duration = sum(meeting_data['Duration'].tolist())/60

  calendar_data['Attendance'][index] = attendance
  calendar_data['Duration'][index] = duration
  calendar_data['User Type'][index] = 'Mentee' if meeting['Participant Identifier'] in mentee_computer_emails_list else 'Mentor'


#FINDING MENTOR THAT MISSED LAST 2 OR 3 SESSIONS

In [590]:
#Identify mentors that have missed 2 or 3 of their last sessions. 

In [591]:
calendar_data['Missed_2'] = [0]*len(calendar_data)
calendar_data['Missed_3'] = [0]*len(calendar_data)

In [592]:
for index, meeting in calendar_data.iterrows():

  #Get previous meetings regarless of id
  previous_meetings = calendar_data[(calendar_data['Event Start UTC'] <= meeting['Event Start UTC']) & (calendar_data['Participant Identifier'] == meeting['Participant Identifier'])] # & (calendar_data['Calendar event ID'] == meeting['Calendar event ID'])]

  if (len(previous_meetings)>=3):
    last_3 = 0 if sum(previous_meetings['Attendance'].tolist()[-3:]) else 1
    last_2 = 0 if sum(previous_meetings['Attendance'].tolist()[-2:]) else 1
  elif (len(previous_meetings) == 2):
    last_3 = 0
    last_2 = 0 if sum(previous_meetings['Attendance'].tolist()) else 1
  else:
    last_3 = 0
    last_2 = 0
  
  #updating data frame 
  calendar_data['Missed_2'][index] = last_2
  calendar_data['Missed_3'][index] = last_3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [593]:
calendar_data.shape

(8065, 17)

In [594]:
#TODO: merge actor names to calendar_data based on participant identifier
# actor_names = google_data[['Participant Identifier', 'Actor name']].drop_duplicates(subset = ['Participant Identifier'])
# print(actor_names.shape)
# actor_names = calendar_data[['Participant Identifier']].drop_duplicates(subset = ['Participant Identifier'])
# print(actor_names.shape)

# #Merge actor names
# result = calendar_data#pd.merge(calendar_data, actor_names, on='Participant Identifier')

# STORING ATTENDANCE DATA

In [595]:
#Create data frame and store
file_name = 'attendance_data_calendar_data.csv'
calendar_data.to_csv(file_name, index = False)

#FINDING PRODUCTIVE MEETINGS

In [596]:
calendar_data.head()

,Program Name,Event Title,Event Start UTC,Event End UTC,Calculated Duration,UTC Day,Guest Status,Created By,Recurring Event,Calendar event ID,Participant Identifier,String_Date,Attendance,Duration,User Type,Missed_2,Missed_3
0,"High Tech, Uganda MP",Test,2022-02-03 16:30:00+00:00,2022-02-03 17:00:00+00:00,0.5,4,INVITED,mentor@villagebookbuilders.org,False,743lcsan6r6p4uaoamu069a275,tyclark25@gmail.com,02/03/2022,1,0,Mentor,0,0
1,"High Tech, Uganda MP",Ivy Gray - VBB Mentoring Session,2022-02-04 14:00:00+00:00,2022-02-04 15:00:00+00:00,1.0,5,YES,mentor@villagebookbuilders.org,False,1f9mq0j68l0ls0supggblr9gel,ivy.gray@jobnimbus.com,02/04/2022,1,45,Mentor,0,0
2,"High Tech, Uganda MP",Ivy Gray - VBB Mentoring Session,2022-02-04 14:00:00+00:00,2022-02-04 15:00:00+00:00,1.0,5,INVITED,mentor@villagebookbuilders.org,False,1f9mq0j68l0ls0supggblr9gel,joshua.atkinson@villagebookbuilders.org,02/04/2022,0,0,Mentor,1,0
3,"High Tech, Uganda MP",Ivy Gray - VBB Mentoring Session,2022-02-04 14:00:00+00:00,2022-02-04 15:00:00+00:00,1.0,5,YES,mentor@villagebookbuilders.org,False,1f9mq0j68l0ls0supggblr9gel,mphightech@villagebookbuilders.org,02/04/2022,0,0,Mentee,0,0
4,"High Tech, Uganda MP",Ivy Gray - VBB Mentoring Session,2022-02-04 14:00:00+00:00,2022-02-04 15:00:00+00:00,1.0,5,INVITED,mentor@villagebookbuilders.org,False,1f9mq0j68l0ls0supggblr9gel,hightech1@villagebookbuilders.org,02/04/2022,1,76,Mentee,0,0


In [597]:
#A productive meeting will be a meeting in which at least one mentor and one mentee attended

In [598]:
#Get all meetings per day 
productive_meetings  = calendar_data[['String_Date', 'Calendar event ID', 'Program Name']].drop_duplicates(subset=['String_Date', 'Calendar event ID'])
productive_meetings['Productive'] = [0]*len(productive_meetings)

In [599]:
#For each meeting in each day see if the meeting was productive

for index, meeting in productive_meetings.iterrows():
  
  mentor_attendance = sum(calendar_data[(calendar_data['String_Date'] == meeting['String_Date']) & (calendar_data['Calendar event ID'].str.contains(meeting['Calendar event ID'])) & (calendar_data['User Type'] == 'Mentor')]['Attendance'].tolist())
  mentee_attendance = sum(calendar_data[(calendar_data['String_Date'] == meeting['String_Date']) & (calendar_data['Calendar event ID'].str.contains(meeting['Calendar event ID'])) & (calendar_data['User Type'] == 'Mentee')]['Attendance'].tolist())

  if(mentor_attendance and mentee_attendance):
    productive_meetings['Productive'][index] = 1
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


# STORING PRODUCTIVE MEETINGS

In [600]:
#Create data frame and store
file_name = 'productive_meetings.csv'
productive_meetings.to_csv(file_name, index = False)

In [601]:
calendar_data.head()

,Program Name,Event Title,Event Start UTC,Event End UTC,Calculated Duration,UTC Day,Guest Status,Created By,Recurring Event,Calendar event ID,Participant Identifier,String_Date,Attendance,Duration,User Type,Missed_2,Missed_3
0,"High Tech, Uganda MP",Test,2022-02-03 16:30:00+00:00,2022-02-03 17:00:00+00:00,0.5,4,INVITED,mentor@villagebookbuilders.org,False,743lcsan6r6p4uaoamu069a275,tyclark25@gmail.com,02/03/2022,1,0,Mentor,0,0
1,"High Tech, Uganda MP",Ivy Gray - VBB Mentoring Session,2022-02-04 14:00:00+00:00,2022-02-04 15:00:00+00:00,1.0,5,YES,mentor@villagebookbuilders.org,False,1f9mq0j68l0ls0supggblr9gel,ivy.gray@jobnimbus.com,02/04/2022,1,45,Mentor,0,0
2,"High Tech, Uganda MP",Ivy Gray - VBB Mentoring Session,2022-02-04 14:00:00+00:00,2022-02-04 15:00:00+00:00,1.0,5,INVITED,mentor@villagebookbuilders.org,False,1f9mq0j68l0ls0supggblr9gel,joshua.atkinson@villagebookbuilders.org,02/04/2022,0,0,Mentor,1,0
3,"High Tech, Uganda MP",Ivy Gray - VBB Mentoring Session,2022-02-04 14:00:00+00:00,2022-02-04 15:00:00+00:00,1.0,5,YES,mentor@villagebookbuilders.org,False,1f9mq0j68l0ls0supggblr9gel,mphightech@villagebookbuilders.org,02/04/2022,0,0,Mentee,0,0
4,"High Tech, Uganda MP",Ivy Gray - VBB Mentoring Session,2022-02-04 14:00:00+00:00,2022-02-04 15:00:00+00:00,1.0,5,INVITED,mentor@villagebookbuilders.org,False,1f9mq0j68l0ls0supggblr9gel,hightech1@villagebookbuilders.org,02/04/2022,1,76,Mentee,0,0


# AVAILABLE PROGRAM SLOTS 

In [602]:
#get total sessions by program
total_sessions = oracle_data['NAME'].value_counts()
total_sessions

TypeError: ignored

In [ ]:
#get count of active sessions
#taken_sessions = active_sessions['NAME'].value_counts()
#taken_sessions

In [ ]:
# available_sessions = total_sessions - taken_sessions
# available_sessions